In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.core import Dropout
%matplotlib inline

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/test.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/submission.csv')

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24414 entries, 0 to 24413
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              24414 non-null  int64  
 1   Country_Region  24414 non-null  object 
 2   Province_State  10374 non-null  object 
 3   Date            24414 non-null  object 
 4   ConfirmedCases  24414 non-null  float64
 5   Fatalities      24414 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ForecastId      13459 non-null  int64 
 1   Country_Region  13459 non-null  object
 2   Province_State  5719 non-null   object
 3   Date            13459 non-null  object
dtypes: int64(1), object(3)
memory usage: 420.7+ KB


In [10]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13459 entries, 0 to 13458
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   ForecastId      13459 non-null  int64
 1   ConfirmedCases  13459 non-null  int64
 2   Fatalities      13459 non-null  int64
dtypes: int64(3)
memory usage: 315.6 KB


In [0]:
train['Country/Province'] = np.where(train['Province_State'].isna() == False, train['Country_Region'] + '/' + train['Province_State'], train['Country_Region'])
test['Country/Province'] = np.where(test['Province_State'].isna() == False, test['Country_Region'] + '/' + test['Province_State'], test['Country_Region'])

In [19]:
countries = train['Country/Province'].unique()
submission=pd.DataFrame(columns=submission.columns)
for country in countries:
    train_df = train[train['Country/Province'] == 'India']
    test_df = test[test['Country/Province'] == 'India']

    X_train_conf = train_df['ConfirmedCases'].values

    scaler_conf = MinMaxScaler()
    scaler_conf.fit(X_train_conf.reshape((-1,1)))
    scaled_train_conf = scaler_conf.transform(X_train_conf.reshape((-1,1)))


    n_input_conf = 60
    generator_conf = TimeseriesGenerator(scaled_train_conf, scaled_train_conf, length=n_input_conf, batch_size=1)


    model_conf = Sequential()
    model_conf.add(LSTM(100, activation='relu',return_sequences=True, input_shape=(n_input_conf,1)))
    model_conf.add(LSTM(100, activation='relu'))
    model_conf.add(Dense(1))
    model_conf.compile(optimizer='adam', loss='mse')
    model_conf.fit_generator(generator_conf,epochs=100)


    test_predictions_conf = []
    first_eval_batch = scaled_train_conf[-(n_input_conf+7):-7]
    current_batch = first_eval_batch.reshape((1, n_input_conf, 1))
    for i in range(len(test_df)):
        current_pred = model_conf.predict(current_batch)[0]
        test_predictions_conf.append(current_pred) 
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)


    
    X_train_fat = train_df[['ConfirmedCases','Fatalities']].values

    scaler_fat = MinMaxScaler()
    scaler_fat.fit(X_train_fat.reshape((-1,2)))
    scaled_train_fat = scaler_fat.transform(X_train_fat.reshape((-1,2)))

    n_input_fat = 30
    generator_fat = TimeseriesGenerator(scaled_train_fat, scaled_train_fat[:,1], length=n_input_fat, batch_size=1)

    model_fat = Sequential()
    model_fat.add(LSTM(100, activation='relu',return_sequences=True, input_shape=(n_input_fat,2)))
    model_fat.add(LSTM(100, activation='relu'))
    model_fat.add(Dense(1))
    model_fat.compile(optimizer='adam', loss='mse')
    model_fat.fit_generator(generator_fat,epochs=100)

    test_predictions_fat = []
    first_eval_batch = scaled_train_fat[-(n_input_fat+7):-7]
    current_batch = first_eval_batch.reshape((1, n_input_fat, 2))
    for i in range(len(test_df)):
        current_pred = model_fat.predict(current_batch)[0]
        test_predictions_fat.append(current_pred) 
        current_batch = np.append(current_batch[:,1:,:],[[[test_predictions_fat[i][0],current_pred]]],axis=1)


    predictions = test_predictions_fat.copy()
    predictions.extend(test_predictions_fat)
    predictions = np.array(predictions).reshape((len(test_df),2),order='F')
    true_predictions = scaler_fat.inverse_transform(predictions).astype(int)

    print(true_predictions)
    test_id=test_df['ForecastId'].values.tolist()
    test_res=pd.DataFrame(columns=submission.columns)
    test_res['ConfirmedCases']=true_predictions[:,0]
    test_res['ForecastId']=test_id
    test_res['Fatalities']=true_predictions[:,1]
    submission=submission.append(test_res)

Epoch 1/100
18/18 [==============================] - 1s 81ms/step - loss: 0.1674
Epoch 2/100
18/18 [==============================] - 1s 48ms/step - loss: 0.0688
Epoch 3/100
18/18 [==============================] - 1s 48ms/step - loss: 0.0554
Epoch 4/100
18/18 [==============================] - 1s 49ms/step - loss: 0.0472
Epoch 5/100
18/18 [==============================] - 1s 48ms/step - loss: 0.0312
Epoch 6/100
18/18 [==============================] - 1s 50ms/step - loss: 0.0132
Epoch 7/100
18/18 [==============================] - 1s 48ms/step - loss: 0.0116
Epoch 8/100
18/18 [==============================] - 1s 52ms/step - loss: 0.0039
Epoch 9/100
18/18 [==============================] - 1s 49ms/step - loss: 0.0030
Epoch 10/100
18/18 [==============================] - 1s 47ms/step - loss: 0.0031
Epoch 11/100
18/18 [==============================] - 1s 49ms/step - loss: 0.0177
Epoch 12/100
18/18 [==============================] - 1s 48ms/step - loss: 0.0061
Epoch 13/100
18/18 [=====

KeyboardInterrupt: ignored

In [14]:
submission

,ForecastId,ConfirmedCases,Fatalities
0,1,155,4
1,2,168,5
2,3,181,5
3,4,194,6
4,5,208,6
5,6,224,7
6,7,242,7
7,8,264,8
8,9,290,9
9,10,321,10
